# Analyzing PPP Loans Data for Bay Area

*Based on ZIP Codes*

In [73]:
import pandas as pd
import numpy as np
import re
import warnings
import os
from os.path import dirname, abspath

warnings.filterwarnings('ignore')

In [74]:
# Get Current Working Directory and Parent Path (for reading files in different folders)
## Source: https://stackoverflow.com/questions/30218802/get-parent-of-current-directory-from-python-script/30218825
d = dirname(dirname(abspath(os.getcwd())))
d

'C:\\Users\\Trang\\Desktop\\afn'

## Part 1: Load Other Tables for PPP Loan Data

The following datasets are external to the PPP Loan Datasets and will be used to match and filter some columns in the PPP Loan files.

`NAICS_codes`: To merge with `PPP_combined` on 6-digit NAICS code, need to match with 4-digit code

`bay_area_zip`: New dataset with zip codes of all of Bay Area (might need to filter for 6 cities)

In [75]:
#Import NAICS 6 and 4 digit concordance table

NAICS_codes = pd.read_csv(d + '/ppp-loan-data/naics/naics_data_rsei_v238.csv')
NAICS_codes['2017NAICSCode'] = NAICS_codes['2017NAICSCode'].apply(str)
NAICS_codes.head()

2017NAICSCode                           LongName Changed2017  \
0        111110                    Soybean Farming         NaN   
1        111120  Oilseed (except Soybean) Farming          NaN   
2        111130          Dry Pea and Bean Farming          NaN   
3        111140                      Wheat Farming         NaN   
4        111150                      Corn Farming          NaN   

  TRIIndustrySector               IndustrySubsector  4DigitNAICS  NewIndustry  
0         999 Other  1111 Oilseed and Grain Farming         1111        False  
1         999 Other  1111 Oilseed and Grain Farming         1111        False  
2         999 Other  1111 Oilseed and Grain Farming         1111        False  
3         999 Other  1111 Oilseed and Grain Farming         1111        False  
4         999 Other  1111 Oilseed and Grain Farming         1111        False

In [76]:
#Import Bay Area Zip Codes (New Dataset as of 5/28/2021) to filter from PPP Loan data
bay_area_zip = pd.read_csv(d + '/ppp-loan-data/bay_zipcodes/cb_2018_us_zcta510_500k_BAY.csv')
bay_area_zip = bay_area_zip.rename(columns={'ZCTA5CE10':'zip_code_5'})
bay_area_zip['zip_code_5'] =bay_area_zip['zip_code_5'].apply(str)
bay_area_zip.head()

zip_code_5      AFFGEOID10  GEOID10    ALAND10  AWATER10
0      94952  8600000US94952    94952  466821663   2998248
1      94127  8600000US94127    94127    4585722      9359
2      95363  8600000US95363    95363  305162606   1236641
3      95441  8600000US95441    95441  263937111   4704473
4      94574  8600000US94574    94574  330686479   5409337

## Part 2: Sample Analysis on one CSV file

Used `public_up_to_150k_1.csv` to:
1. Analyze NaN values in PPP Loans data. *(See Questions and Notes)
2. Split full dataframe in 2: NaN Zip Codes & Non-NaN Zip Codes
3. Added modified zip code column to Non-NULL dataframe
4. Extracted only Bay Area Zip Codes using table in Part 1

**TODO:** 
- Analyze NaN Zips and add Bay Area & Minority-Owned ones to `bay_area_ppp`
- Merge the NAICS 4-digit codes
- Apply analysis to all files to make `PPP_combined`

In [3]:
# Read in one file
ppp_sub_150_1 = pd.read_csv(d + '/ppp-loan-data/ppp_loan_datasets/public_up_to_150k_1.csv')
ppp_sub_150_1.head()

LoanNumber DateApproved  SBAOfficeCode ProcessingMethod  \
0  5375617707   05/01/2020          101.0              PPP   
1  9677497701   05/01/2020          464.0              PPP   
2  9547167709   05/01/2020          464.0              PPP   
3  6448037706   05/01/2020          515.0              PPP   
4  9609017706   05/01/2020          464.0              PPP   

                                     BorrowerName          BorrowerAddress  \
0                                   NOT AVAILABLE                      NaN   
1          NORTH CHARLESTON HOSPITALITY GROUP LLC      192 College Park Rd   
2                            Q AND J SERVICES LLC  301 Old Georgetown Road   
3  OPTIMIZED PROCESS SOLUTIONS DBA AAA INDUSTRIES            24500 CAPITOL   
4                            D2 POWER SPORTS, LLC           125 Simuel Dr.   

  BorrowerCity BorrowerState BorrowerZip LoanStatusDate  ...  \
0          NaN           NaN         NaN            NaN  ...   
1       Ladson           NaN  29456-3517            NaN  ...   
2      Manning           NaN  29102-2734     04/20/2021  ...   
3      REDFORD           NaN  48239-2446     04/16/2021  ...   
4  Spartanburg           NaN  29303-2085            NaN  ...   

  HEALTH_CARE_PROCEED  DEBT_INTEREST_PROCEED                     BusinessType  \
0                 NaN                    NaN                              NaN   
1                 NaN                    NaN  Limited  Liability Company(LLC)   
2                 NaN                    NaN  Limited  Liability Company(LLC)   
3                 NaN                    NaN  Limited  Liability Company(LLC)   
4                 NaN                    NaN  Limited  Liability Company(LLC)   

   OriginatingLenderLocationID                      OriginatingLender  \
0                       9551.0  Bank of America, National Association   
1                      19248.0                           Synovus Bank   
2                      19248.0                           Synovus Bank   
3                       9551.0  Bank of America, National Association   
4                      19248.0                           Synovus Bank   

   OriginatingLenderCity OriginatingLenderState      Gender      Veteran  \
0              CHARLOTTE                     NC  Unanswered   Unanswered   
1               COLUMBUS                     GA  Unanswered   Unanswered   
2               COLUMBUS                     GA  Unanswered   Unanswered   
3              CHARLOTTE                     NC  Male Owned  Non-Veteran   
4               COLUMBUS                     GA  Unanswered   Unanswered   

  NonProfit  
0       NaN  
1       NaN  
2       NaN  
3       NaN  
4       NaN  

[5 rows x 51 columns]

### Part 2a. Analyze NaN Zip Codes

Looking Into Businesses with little to no information on location

**Questions:**

1. What should we do with NaN zip codes? What if it is minority-owned but area is unknown?

**Notes:**

There are 124 NaN Zip Codes in `ppp_sub_150_1`. Should we drop them all? Or manually filter them?

Idea - Automate Process:

1. In each table, get all NaN Zip Codes -> New DF
2. In new DF, get Race and Ethnicity where it is answered
3. Filter for anything !White and !Not Hispanic or Latino
4. View DF with indices of minority businesses with no Zip Codes -> Manually Check?
5. If 'BorrowerState' or 'ProjectState' == CA: keep, else: drop from table
6. Manually check remaining rows.

In [19]:
# NaN Example: No location information, Nothing on Race and Ethincity
ppp_sub_150_1.loc[0]

LoanNumber                                                5375617707
DateApproved                                              05/01/2020
SBAOfficeCode                                                    101
ProcessingMethod                                                 PPP
BorrowerName                                           NOT AVAILABLE
BorrowerAddress                                                  NaN
BorrowerCity                                                     NaN
BorrowerState                                                    NaN
BorrowerZip                                                      NaN
LoanStatusDate                                                   NaN
LoanStatus                                               Exemption 4
Term                                                              24
SBAGuarantyPercentage                                            100
InitialApprovalAmount                                         148440
CurrentApprovalAmount             

In [17]:
# Count NaN Zip Codes (BorrowerZip) in table
ppp_sub_150_1['BorrowerZip'].isna().sum()

124

In [28]:
# Get rows where BorrowerZip is NaN
nan_zip_df = ppp_sub_150_1[ppp_sub_150_1['BorrowerZip'].isnull()]
nan_zip_df.head()

LoanNumber DateApproved  SBAOfficeCode ProcessingMethod  \
0  5375617707   05/01/2020          101.0              PPP   
5  9789867710   05/01/2020          101.0              PPP   
6  9589997709   05/01/2020          101.0              PPP   
8  9662387700   05/01/2020          101.0              PPP   
9  2767027201   04/16/2020            NaN              PPP   

                       BorrowerName BorrowerAddress BorrowerCity  \
0                     NOT AVAILABLE             NaN          NaN   
5      VULCAN MACHINERY CORPORATION             NaN          NaN   
6  TJK KITCHENS &AMP; BREWPUBS, LLC             NaN          NaN   
8                     RON GOLDSTONE             NaN          NaN   
9                       Exemption 6             NaN          NaN   

  BorrowerState BorrowerZip LoanStatusDate  ... HEALTH_CARE_PROCEED  \
0           NaN         NaN            NaN  ...                 NaN   
5           NaN         NaN            NaN  ...                 NaN   
6           NaN         NaN            NaN  ...                 NaN   
8           NaN         NaN            NaN  ...                 NaN   
9           NaN         NaN     02/18/2021  ...                 0.0   

   DEBT_INTEREST_PROCEED                     BusinessType  \
0                    NaN                              NaN   
5                    NaN                      Corporation   
6                    NaN  Limited  Liability Company(LLC)   
8                    NaN                              NaN   
9                    0.0         Subchapter S Corporation   

   OriginatingLenderLocationID                      OriginatingLender  \
0                       9551.0  Bank of America, National Association   
5                      57328.0           The Huntington National Bank   
6                      57328.0           The Huntington National Bank   
8                      57328.0           The Huntington National Bank   
9                          NaN                                    NaN   

   OriginatingLenderCity OriginatingLenderState      Gender      Veteran  \
0              CHARLOTTE                     NC  Unanswered   Unanswered   
5               COLUMBUS                     OH  Male Owned  Non-Veteran   
6               COLUMBUS                     OH  Unanswered   Unanswered   
8               COLUMBUS                     OH  Unanswered   Unanswered   
9                    NaN                    NaN  Unanswered   Unanswered   

  NonProfit  
0       NaN  
5       NaN  
6       NaN  
8       NaN  
9       NaN  

[5 rows x 51 columns]

In [31]:
# Get Race of these NaN rows
nan_zip_df[nan_zip_df['Race'] != 'Unanswered']['Race']

11                            White
15                            White
22                            White
25                            White
26                            White
48                            White
51                            White
67        Black or African American
76                            White
79                            White
83                            White
87                            White
95                            White
102                           White
103                           White
119                           White
120                           White
130                           White
466347                        Asian
Name: Race, dtype: object

In [32]:
# Get Ethnicity of these NaN rows
nan_zip_df[nan_zip_df['Ethnicity'] != 'Unknown/NotStated']['Ethnicity']

11        Not Hispanic or Latino
22        Not Hispanic or Latino
25        Not Hispanic or Latino
26        Not Hispanic or Latino
48        Not Hispanic or Latino
51        Not Hispanic or Latino
67        Not Hispanic or Latino
76        Not Hispanic or Latino
79        Not Hispanic or Latino
83        Not Hispanic or Latino
87        Not Hispanic or Latino
95        Not Hispanic or Latino
102       Not Hispanic or Latino
103       Not Hispanic or Latino
119       Not Hispanic or Latino
120       Not Hispanic or Latino
130       Not Hispanic or Latino
133           Hispanic or Latino
466347    Not Hispanic or Latino
Name: Ethnicity, dtype: object

***Examples of NaN, Minority-owned businesses***

In [39]:
'''
BorrowerName                                 SUNKISSBIZ LLC
BorrowerAddress                                         NaN
BorrowerCity                                            NaN
BorrowerState                                           NaN
BorrowerZip                                             NaN
...
NAICSCode                                            611610
Race                              Black or African American
Ethnicity                            Not Hispanic or Latino
'''
nan_zip_df.loc[67] #Located in MI -> Drop

In [40]:
'''
BorrowerName                                       ART N FUN STUDIO, INC
BorrowerAddress                                                      NaN
BorrowerCity                                                         NaN
BorrowerState                                                         CA
BorrowerZip                                                          NaN
...
NAICSCode                                                         712110
Race                                                               Asian
Ethnicity                                         Not Hispanic or Latino
'''
nan_zip_df.loc[466347] # Located In Santa Clara, CA, 95051 -> Keep?

In [41]:
'''
BorrowerName                           Exemption 6
BorrowerAddress                                NaN
BorrowerCity                                   NaN
BorrowerState                                  NaN
BorrowerZip                                    NaN
...
NAICSCode                                   561720
Race                                    Unanswered
Ethnicity                       Hispanic or Latino
'''
nan_zip_df.loc[133] # No location information

### Part 2b. Split Dataframe - Get Non-NaN Dataframe 


In [88]:
not_nan_df = ppp_sub_150_1[ppp_sub_150_1['BorrowerZip'].notnull()]

not_nan_df.head()

LoanNumber DateApproved  SBAOfficeCode ProcessingMethod  \
1  9677497701   05/01/2020          464.0              PPP   
2  9547167709   05/01/2020          464.0              PPP   
3  6448037706   05/01/2020          515.0              PPP   
4  9609017706   05/01/2020          464.0              PPP   
7  6486007709   05/01/2020          914.0              PPP   

                                     BorrowerName            BorrowerAddress  \
1          NORTH CHARLESTON HOSPITALITY GROUP LLC        192 College Park Rd   
2                            Q AND J SERVICES LLC    301 Old Georgetown Road   
3  OPTIMIZED PROCESS SOLUTIONS DBA AAA INDUSTRIES              24500 CAPITOL   
4                            D2 POWER SPORTS, LLC             125 Simuel Dr.   
7                               SEAN T KY DDS INC  200 S EL MOLINO AVE STE 5   

  BorrowerCity BorrowerState BorrowerZip LoanStatusDate  ...  \
1       Ladson           NaN  29456-3517            NaN  ...   
2      Manning           NaN  29102-2734     04/20/2021  ...   
3      REDFORD           NaN  48239-2446     04/16/2021  ...   
4  Spartanburg           NaN  29303-2085            NaN  ...   
7     PASADENA           NaN  91101-2985            NaN  ...   

  HEALTH_CARE_PROCEED  DEBT_INTEREST_PROCEED                     BusinessType  \
1                 NaN                    NaN  Limited  Liability Company(LLC)   
2                 NaN                    NaN  Limited  Liability Company(LLC)   
3                 NaN                    NaN  Limited  Liability Company(LLC)   
4                 NaN                    NaN  Limited  Liability Company(LLC)   
7                 NaN                    NaN                      Corporation   

   OriginatingLenderLocationID                      OriginatingLender  \
1                      19248.0                           Synovus Bank   
2                      19248.0                           Synovus Bank   
3                       9551.0  Bank of America, National Association   
4                      19248.0                           Synovus Bank   
7                       9551.0  Bank of America, National Association   

   OriginatingLenderCity OriginatingLenderState      Gender      Veteran  \
1               COLUMBUS                     GA  Unanswered   Unanswered   
2               COLUMBUS                     GA  Unanswered   Unanswered   
3              CHARLOTTE                     NC  Male Owned  Non-Veteran   
4               COLUMBUS                     GA  Unanswered   Unanswered   
7              CHARLOTTE                     NC  Unanswered   Unanswered   

  NonProfit  
1       NaN  
2       NaN  
3       NaN  
4       NaN  
7       NaN  

[5 rows x 51 columns]

In [89]:
# Check that lengths of DF split matches up
len(ppp_sub_150_1) == len(not_nan_df) + len(nan_zip_df)

True

### Part 2c. Add Modified Zip Code Column

Extract first 5 digits of `BorrowerZip` column

In [94]:
# Source: https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
pattern = '(^\d{5})'
not_nan_df['zip_code_5'] = not_nan_df['BorrowerZip'].str.extract(pattern)
not_nan_df.head()

C:\Users\Trang\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


LoanNumber DateApproved  SBAOfficeCode ProcessingMethod  \
1  9677497701   05/01/2020          464.0              PPP   
2  9547167709   05/01/2020          464.0              PPP   
3  6448037706   05/01/2020          515.0              PPP   
4  9609017706   05/01/2020          464.0              PPP   
7  6486007709   05/01/2020          914.0              PPP   

                                     BorrowerName            BorrowerAddress  \
1          NORTH CHARLESTON HOSPITALITY GROUP LLC        192 College Park Rd   
2                            Q AND J SERVICES LLC    301 Old Georgetown Road   
3  OPTIMIZED PROCESS SOLUTIONS DBA AAA INDUSTRIES              24500 CAPITOL   
4                            D2 POWER SPORTS, LLC             125 Simuel Dr.   
7                               SEAN T KY DDS INC  200 S EL MOLINO AVE STE 5   

  BorrowerCity BorrowerState BorrowerZip LoanStatusDate  ...  \
1       Ladson           NaN  29456-3517            NaN  ...   
2      Manning           NaN  29102-2734     04/20/2021  ...   
3      REDFORD           NaN  48239-2446     04/16/2021  ...   
4  Spartanburg           NaN  29303-2085            NaN  ...   
7     PASADENA           NaN  91101-2985            NaN  ...   

  DEBT_INTEREST_PROCEED                     BusinessType  \
1                   NaN  Limited  Liability Company(LLC)   
2                   NaN  Limited  Liability Company(LLC)   
3                   NaN  Limited  Liability Company(LLC)   
4                   NaN  Limited  Liability Company(LLC)   
7                   NaN                      Corporation   

   OriginatingLenderLocationID                      OriginatingLender  \
1                      19248.0                           Synovus Bank   
2                      19248.0                           Synovus Bank   
3                       9551.0  Bank of America, National Association   
4                      19248.0                           Synovus Bank   
7                       9551.0  Bank of America, National Association   

   OriginatingLenderCity  OriginatingLenderState      Gender      Veteran  \
1               COLUMBUS                      GA  Unanswered   Unanswered   
2               COLUMBUS                      GA  Unanswered   Unanswered   
3              CHARLOTTE                      NC  Male Owned  Non-Veteran   
4               COLUMBUS                      GA  Unanswered   Unanswered   
7              CHARLOTTE                      NC  Unanswered   Unanswered   

  NonProfit zip_code_5  
1       NaN      29456  
2       NaN      29102  
3       NaN      48239  
4       NaN      29303  
7       NaN      91101  

[5 rows x 52 columns]

In [84]:
bay_area_ppp = not_nan_df[not_nan_df['zip_code_5'].isin(bay_area_zip['zip_code_5'])]
bay_area_ppp.head()

LoanNumber DateApproved  SBAOfficeCode ProcessingMethod  \
102272  8585717704   05/01/2020          459.0              PPP   
392350  1464468601   03/13/2021          912.0              PPS   
392359  2663308509   02/22/2021          912.0              PPS   
392364  4137348607   03/18/2021          912.0              PPS   
392365  4315748602   03/18/2021          912.0              PPS   

                                   BorrowerName           BorrowerAddress  \
102272  MARKET AUTO TRUCK COLLISION CORPORATION          140 SAN JOSE AVE   
392350                GOLD RUSH KETTLE KORN LLC       4690 E 2nd St Ste 9   
392359                         KALIBER LABS INC      188 King St Unit 307   
392364                ROSS MCDONALD COMPANY INC           1154 Stealth St   
392365                   GATES EISENHART DAWSON  125 S Market St Ste 1200   

         BorrowerCity BorrowerState BorrowerZip LoanStatusDate  ...  \
102272       SAN JOSE            AL       95125            NaN  ...   
392350        Benicia            CA  94510-1008            NaN  ...   
392359  San Francisco            CA  94107-4903            NaN  ...   
392364      Livermore            CA  94551-9300            NaN  ...   
392365       San Jose            CA  95113-2288            NaN  ...   

       DEBT_INTEREST_PROCEED                     BusinessType  \
102272                   NaN                      Corporation   
392350                   NaN  Limited  Liability Company(LLC)   
392359                   NaN                      Corporation   
392364                   NaN                      Corporation   
392365                   NaN                      Partnership   

        OriginatingLenderLocationID                      OriginatingLender  \
102272                     122043.0                                WebBank   
392350                      11822.0  MUFG Union Bank, National Association   
392359                       9551.0  Bank of America, National Association   
392364                     474333.0                    First Republic Bank   
392365                     474333.0                    First Republic Bank   

        OriginatingLenderCity  OriginatingLenderState      Gender  \
102272         SALT LAKE CITY                      UT  Unanswered   
392350          SAN FRANCISCO                      CA  Male Owned   
392359              CHARLOTTE                      NC  Male Owned   
392364          SAN FRANCISCO                      CA  Unanswered   
392365          SAN FRANCISCO                      CA  Unanswered   

            Veteran NonProfit zip_code_5  
102272   Unanswered       NaN      95125  
392350  Non-Veteran       NaN      94510  
392359  Non-Veteran       NaN      94107  
392364   Unanswered       NaN      94551  
392365  Non-Veteran       NaN      95113  

[5 rows x 52 columns]

In [96]:
#Create new column for year of approval 
bay_area_ppp.loc[:,'YearApproved'] = bay_area_ppp['DateApproved'].str[-4:]

#Create new column for NAICS 6 digit in string format 
bay_area_ppp.loc[:,'NAICS_6'] = bay_area_ppp['NAICSCode'].apply(lambda y: str(y)[:6])

### Part 2d. Add `Minority` Column 

Options: Yes, No, Unanswered

In [118]:
bay_area_ppp[(bay_area_ppp['Ethnicity'] == 'Hispanic or Latino')]['Race']

# Question: Hispanic or Latino +  White? => Yes or No

392397                        White
392410                        White
392530                        White
392858                        White
392868                        White
                    ...            
899256                   Unanswered
899439                   Unanswered
899733    Black or African American
899775                        White
899949                        White
Name: Race, Length: 3274, dtype: object

In [120]:
# Source: https://www.dataquest.io/blog/tutorial-add-column-pandas-dataframe-based-on-if-else-condition/

conditions = [
    (bay_area_ppp['Race']  == 'White'),
    (bay_area_ppp['Race'] == 'Unanswered') & (bay_area_ppp['Ethnicity'] == 'Not Hispanic or Latino'),
    (bay_area_ppp['Race'] == 'Unanswered') & (bay_area_ppp['Ethnicity'] == 'Unknown/NotStated'),
    (bay_area_ppp['Ethnicity'] == 'Hispanic or Latino') ,
    (bay_area_ppp['Race']  != 'White') & (bay_area_ppp['Race'] != 'Unanswered') ,
    ]

values = ['No', 'No', 'Unanswered', 'Yes', 'Yes']

# Note: 0 in Minority column indicates a condition that is not covered
bay_area_ppp['Minority'] = np.select(conditions, values)
bay_area_ppp.head()

LoanNumber DateApproved  SBAOfficeCode ProcessingMethod  \
102272  8585717704   05/01/2020          459.0              PPP   
392350  1464468601   03/13/2021          912.0              PPS   
392359  2663308509   02/22/2021          912.0              PPS   
392364  4137348607   03/18/2021          912.0              PPS   
392365  4315748602   03/18/2021          912.0              PPS   

                                   BorrowerName           BorrowerAddress  \
102272  MARKET AUTO TRUCK COLLISION CORPORATION          140 SAN JOSE AVE   
392350                GOLD RUSH KETTLE KORN LLC       4690 E 2nd St Ste 9   
392359                         KALIBER LABS INC      188 King St Unit 307   
392364                ROSS MCDONALD COMPANY INC           1154 Stealth St   
392365                   GATES EISENHART DAWSON  125 S Market St Ste 1200   

         BorrowerCity BorrowerState BorrowerZip LoanStatusDate  ...  \
102272       SAN JOSE            AL       95125            NaN  ...   
392350        Benicia            CA  94510-1008            NaN  ...   
392359  San Francisco            CA  94107-4903            NaN  ...   
392364      Livermore            CA  94551-9300            NaN  ...   
392365       San Jose            CA  95113-2288            NaN  ...   

                            OriginatingLender  OriginatingLenderCity  \
102272                                WebBank         SALT LAKE CITY   
392350  MUFG Union Bank, National Association          SAN FRANCISCO   
392359  Bank of America, National Association              CHARLOTTE   
392364                    First Republic Bank          SAN FRANCISCO   
392365                    First Republic Bank          SAN FRANCISCO   

        OriginatingLenderState      Gender      Veteran  NonProfit zip_code_5  \
102272                      UT  Unanswered   Unanswered        NaN      95125   
392350                      CA  Male Owned  Non-Veteran        NaN      94510   
392359                      NC  Male Owned  Non-Veteran        NaN      94107   
392364                      CA  Unanswered   Unanswered        NaN      94551   
392365                      CA  Unanswered  Non-Veteran        NaN      95113   

        YearApproved NAICS_6    Minority  
102272          2020  811121  Unanswered  
392350          2021  722330          No  
392359          2021  541511          No  
392364          2021  541420  Unanswered  
392365          2021  541110  Unanswered  

[5 rows x 55 columns]

### Part 2e. Add NAICS 4-digit Column

In [126]:
#Merge NAICS 4 digit with PPP data and remove the 4 digit code from IndustrySubsector name
bay_area_ppp_NAICS = pd.merge(bay_area_ppp, NAICS_codes, how='left', left_on='NAICS_6', right_on='2017NAICSCode')
bay_area_ppp_NAICS.loc[:,'NAICS_4'] = bay_area_ppp_NAICS['4DigitNAICS'].apply(lambda y: str(y)[:4])
bay_area_ppp_NAICS = bay_area_ppp_NAICS.drop(['2017NAICSCode','Changed2017','TRIIndustrySector','NewIndustry','4DigitNAICS'], axis=1)
bay_area_ppp_NAICS['IndustrySubsector'] = bay_area_ppp_NAICS['IndustrySubsector'].str[5:]
bay_area_ppp_NAICS.head()

LoanNumber DateApproved  SBAOfficeCode ProcessingMethod  \
0  8585717704   05/01/2020          459.0              PPP   
1  1464468601   03/13/2021          912.0              PPS   
2  2663308509   02/22/2021          912.0              PPS   
3  4137348607   03/18/2021          912.0              PPS   
4  4315748602   03/18/2021          912.0              PPS   

                              BorrowerName           BorrowerAddress  \
0  MARKET AUTO TRUCK COLLISION CORPORATION          140 SAN JOSE AVE   
1                GOLD RUSH KETTLE KORN LLC       4690 E 2nd St Ste 9   
2                         KALIBER LABS INC      188 King St Unit 307   
3                ROSS MCDONALD COMPANY INC           1154 Stealth St   
4                   GATES EISENHART DAWSON  125 S Market St Ste 1200   

    BorrowerCity BorrowerState BorrowerZip LoanStatusDate  ...      Gender  \
0       SAN JOSE            AL       95125            NaN  ...  Unanswered   
1        Benicia            CA  94510-1008            NaN  ...  Male Owned   
2  San Francisco            CA  94107-4903            NaN  ...  Male Owned   
3      Livermore            CA  94551-9300            NaN  ...  Unanswered   
4       San Jose            CA  95113-2288            NaN  ...  Unanswered   

       Veteran  NonProfit  zip_code_5  YearApproved  NAICS_6    Minority  \
0   Unanswered        NaN       95125          2020   811121  Unanswered   
1  Non-Veteran        NaN       94510          2021   722330          No   
2  Non-Veteran        NaN       94107          2021   541511          No   
3   Unanswered        NaN       94551          2021   541420  Unanswered   
4  Non-Veteran        NaN       95113          2021   541110  Unanswered   

                                            LongName  \
0  Automotive Body, Paint, and Interior Repair an...   
1                               Mobile Food Services   
2              Custom Computer Programming Services    
3                         Industrial Design Services   
4                                 Offices of Lawyers   

                              IndustrySubsector NAICS_4  
0             Automotive Repair and Maintenance    8111  
1                         Special Food Services    7223  
2  Computer Systems Design and Related Services    5415  
3                   Specialized Design Services    5414  
4                                Legal Services    5411  

[5 rows x 58 columns]

## Part 3. Go through all files to build `PPP_combined`

In [77]:
def process_files():
    '''
    Apply analysis to all PPP loan data files without loading it all at once.
    1. Split dataframe in 2 based on whether or not BorrowerZip is null.
    2. Add 5-digit Zip Code, Minority, and 4-digit NAICS code columns of ONLY BAY AREA BUSINESSES
    3. Append split and processed dataframes to list for concatentation
    4. Concatenate all dataframes into PPP_BA_combined (contains Bay Area businesses)
    5. Save comprehensive list to CSV files
    
    Returns two Dataframes
    '''
    filenames = [d + '/ppp-loan-data/ppp_loan_datasets/public_up_to_150k_{}.csv'.format(i) for i in range(1,12)]
    filenames.append(d + '/ppp-loan-data/ppp_loan_datasets/public_150k_plus.csv')
    nan_df_list = list()
    PPP_combined_list = list()
    for file in filenames:
        print('Processing {}...'.format(file))
        df = pd.read_csv(file)
                       
        nan_zip_df, not_nan_df = split_df_on_zip(df)
        
        
        # Add Zip Code, Minority, and NAICS_4 columns
        PPP_bay = add_zip_col(not_nan_df)
        PPP_bay = add_minority_col(PPP_bay) 
        PPP_bay = add_naics_4_col(PPP_bay)
        
        # append nan zips from dataframe
        nan_df_list.append(nan_zip_df)
        
        # append ppp_combined
        PPP_combined_list.append(PPP_bay)
    
    print('Finished reading files.')
    # Join dataframes across all files
    print('Joining dataframes...')
    main_nan_zip = pd.concat(nan_df_list)
    PPP_BA_combined = pd.concat(PPP_combined_list)
    
    
    # Save to files
    print('Saving to file...')
    main_nan_zip.to_csv(d + '/ppp-loan-data/out/nan_zips.csv', index=False)
    PPP_BA_combined.to_csv(d + '/ppp-loan-data/out/bay_bus_from_ppp.csv', index=False)
    
    print('done.')
    return main_nan_zip, PPP_BA_combined

In [78]:
def split_df_on_zip(df):
    '''
    Counts the NaN Zip Code rows in a table and creates 2 dataframes based on whether BorrowerZip is null or not.
    Checks that there are no excluded rows in split.
    
    Input: Dataframe - full table of businesses
    Output: Returns two Dataframes split from input
    '''
    # Count NaN Zip Codes (BorrowerZip) in table
    print('There are {} NaN values in this table.'.format(df['BorrowerZip'].isna().sum()))
    
    # Get rows where BorrowerZip is NaN
    nan_zip_df = df[df['BorrowerZip'].isnull()]
    # Get rows where BorrowerZip is not NaN
    not_nan_df = df[df['BorrowerZip'].notnull()]

    # Check that lengths of DF split matches up
    print('Split length matches total:', len(df) == len(not_nan_df) + len(nan_zip_df))
    
    return nan_zip_df, not_nan_df

In [79]:
def add_zip_col(df):
    '''
    Use Regex to extract the 5-digit Zip Code from BorrowerZip column. Create new column
    and matched new column with known Bay Area zip codes from 'cb_2018_us_zcta510_500k_BAY.csv'
    
    Input: Dataframe without null Zip codes
    Output: Updated Dataframe filtered for Bay Area Zip Codes
    '''
    # Source: https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
    pattern = '(^\d{5})'
    df['zip_code_5'] = df['BorrowerZip'].str.extract(pattern)
    ppp_df_with_zip = df[df['zip_code_5'].isin(bay_area_zip['zip_code_5'])]
    
    return ppp_df_with_zip

In [80]:
def add_minority_col(df):
    '''
    Created new column in dataframe with four options based on Race and Ethinicity column:
    {Yes, No, Unanswered}.
    
    Input: Dataframe without null Zip codes
    Output: Updated Dataframe with Minority column.
    '''
    # Source: https://www.dataquest.io/blog/tutorial-add-column-pandas-dataframe-based-on-if-else-condition/
    conditions = [
        (df['Race']  == 'White'),
        (df['Race'] == 'Unanswered') & (df['Ethnicity'] == 'Not Hispanic or Latino'),
        (df['Race'] == 'Unanswered') & (df['Ethnicity'] == 'Unknown/NotStated'),
        (df['Ethnicity'] == 'Hispanic or Latino') ,
        (df['Race']  != 'White') & (df['Race'] != 'Unanswered') ,
        ]

    values = ['No', 'No', 'Unanswered', 'Yes', 'Yes']

    # Note: 0 in Minority column indicates a condition that is not covered
    df.loc[:,'Minority'] = np.select(conditions, values)
    return df

In [81]:
def add_naics_4_col(df):
    '''
    Adde YearApproved and NAICS_6 columns. Merged dataframe with NAICS_codes 
    to remove 4-digit code from IndustrySubsector name.
    
    Input: Dataframe without null Zip codes
    Output: Updated Dataframe with YearApproved, NAICS_6, NAICS_4 column.
    '''
    #Create new column for year of approval 
    df.loc[:,'YearApproved'] = df['DateApproved'].str[-4:]

    #Create new column for NAICS 6 digit in string format 
    df.loc[:,'NAICS_6'] = df['NAICSCode'].apply(lambda y: str(y)[:6])
    
    #Merge NAICS 4 digit with PPP data and remove the 4 digit code from IndustrySubsector name
    bay_area_ppp_NAICS = pd.merge(df, NAICS_codes, how='left', left_on='NAICS_6', right_on='2017NAICSCode')
    bay_area_ppp_NAICS.loc[:,'NAICS_4'] = bay_area_ppp_NAICS['4DigitNAICS'].apply(lambda y: str(y)[:4])
    bay_area_ppp_NAICS = bay_area_ppp_NAICS.drop(['2017NAICSCode','Changed2017','TRIIndustrySector','NewIndustry','4DigitNAICS'], axis=1)
    bay_area_ppp_NAICS.loc[:,'IndustrySubsector'] = bay_area_ppp_NAICS['IndustrySubsector'].str[5:]
    
    return bay_area_ppp_NAICS

In [82]:
# Run to process all the files and get 2 resulting dataframes

nan_zips, PPP_combined = process_files()

Processing C:\Users\Trang\Desktop\afn/ppp-loan-data/ppp_loan_datasets/public_up_to_150k_1.csv...
There are 124 NaN values in this table.
Split length matches total: True
Processing C:\Users\Trang\Desktop\afn/ppp-loan-data/ppp_loan_datasets/public_up_to_150k_2.csv...
There are 8 NaN values in this table.
Split length matches total: True
Processing C:\Users\Trang\Desktop\afn/ppp-loan-data/ppp_loan_datasets/public_up_to_150k_3.csv...
There are 3 NaN values in this table.
Split length matches total: True
Processing C:\Users\Trang\Desktop\afn/ppp-loan-data/ppp_loan_datasets/public_up_to_150k_4.csv...
There are 4 NaN values in this table.
Split length matches total: True
Processing C:\Users\Trang\Desktop\afn/ppp-loan-data/ppp_loan_datasets/public_up_to_150k_5.csv...
There are 4 NaN values in this table.
Split length matches total: True
Processing C:\Users\Trang\Desktop\afn/ppp-loan-data/ppp_loan_datasets/public_up_to_150k_6.csv...
There are 0 NaN values in this table.
Split length matches to

In [49]:
nan_zips.head()

# TODO: Manually go through table to check for Bay Area businesses. Additionally for Minority-owned BA businesses.

LoanNumber DateApproved  SBAOfficeCode ProcessingMethod  \
0  5375617707   05/01/2020          101.0              PPP   
5  9789867710   05/01/2020          101.0              PPP   
6  9589997709   05/01/2020          101.0              PPP   
8  9662387700   05/01/2020          101.0              PPP   
9  2767027201   04/16/2020            NaN              PPP   

                       BorrowerName BorrowerAddress BorrowerCity  \
0                     NOT AVAILABLE             NaN          NaN   
5      VULCAN MACHINERY CORPORATION             NaN          NaN   
6  TJK KITCHENS &AMP; BREWPUBS, LLC             NaN          NaN   
8                     RON GOLDSTONE             NaN          NaN   
9                       Exemption 6             NaN          NaN   

  BorrowerState BorrowerZip LoanStatusDate  ... HEALTH_CARE_PROCEED  \
0           NaN         NaN            NaN  ...                 NaN   
5           NaN         NaN            NaN  ...                 NaN   
6           NaN         NaN            NaN  ...                 NaN   
8           NaN         NaN            NaN  ...                 NaN   
9           NaN         NaN     02/18/2021  ...                 0.0   

   DEBT_INTEREST_PROCEED                     BusinessType  \
0                    NaN                              NaN   
5                    NaN                      Corporation   
6                    NaN  Limited  Liability Company(LLC)   
8                    NaN                              NaN   
9                    0.0         Subchapter S Corporation   

   OriginatingLenderLocationID                      OriginatingLender  \
0                       9551.0  Bank of America, National Association   
5                      57328.0           The Huntington National Bank   
6                      57328.0           The Huntington National Bank   
8                      57328.0           The Huntington National Bank   
9                          NaN                                    NaN   

   OriginatingLenderCity OriginatingLenderState      Gender      Veteran  \
0              CHARLOTTE                     NC  Unanswered   Unanswered   
5               COLUMBUS                     OH  Male Owned  Non-Veteran   
6               COLUMBUS                     OH  Unanswered   Unanswered   
8               COLUMBUS                     OH  Unanswered   Unanswered   
9                    NaN                    NaN  Unanswered   Unanswered   

  NonProfit  
0       NaN  
5       NaN  
6       NaN  
8       NaN  
9       NaN  

[5 rows x 51 columns]

In [83]:
# Add NaN Zips to PPP_combined after manually going through them.
nans_in_bay_list = [466347]
nans_in_bay = nan_zips.loc[nans_in_bay_list]
nans_in_bay['zip_code_5'] = np.nan
nans_in_bay = add_minority_col(nans_in_bay) 
nans_in_bay = add_naics_4_col(nans_in_bay)
nans_in_bay

LoanNumber DateApproved  SBAOfficeCode ProcessingMethod  \
0  2946407704   05/01/2020          912.0              PPP   

            BorrowerName BorrowerAddress BorrowerCity BorrowerState  \
0  ART N FUN STUDIO, INC             NaN          NaN            CA   

  BorrowerZip LoanStatusDate  ...        Gender      Veteran  NonProfit  \
0         NaN            NaN  ...  Female Owned  Non-Veteran        NaN   

   zip_code_5  Minority  YearApproved NAICS_6  LongName  \
0         NaN       Yes          2020  712110  Museums    

                                   IndustrySubsector NAICS_4  
0  Museums, Historical Sites, and Similar Institu...    7121  

[1 rows x 58 columns]

In [84]:
# Append NaN Rows to main dataframe, PPP_combined
PPP_combined = PPP_combined.append(nans_in_bay, ignore_index=True)

In [85]:
PPP_combined.head()

LoanNumber DateApproved  SBAOfficeCode ProcessingMethod  \
0  8585717704   05/01/2020          459.0              PPP   
1  1464468601   03/13/2021          912.0              PPS   
2  2663308509   02/22/2021          912.0              PPS   
3  4137348607   03/18/2021          912.0              PPS   
4  4315748602   03/18/2021          912.0              PPS   

                              BorrowerName           BorrowerAddress  \
0  MARKET AUTO TRUCK COLLISION CORPORATION          140 SAN JOSE AVE   
1                GOLD RUSH KETTLE KORN LLC       4690 E 2nd St Ste 9   
2                         KALIBER LABS INC      188 King St Unit 307   
3                ROSS MCDONALD COMPANY INC           1154 Stealth St   
4                   GATES EISENHART DAWSON  125 S Market St Ste 1200   

    BorrowerCity BorrowerState BorrowerZip LoanStatusDate  ...      Gender  \
0       SAN JOSE            AL       95125            NaN  ...  Unanswered   
1        Benicia            CA  94510-1008            NaN  ...  Male Owned   
2  San Francisco            CA  94107-4903            NaN  ...  Male Owned   
3      Livermore            CA  94551-9300            NaN  ...  Unanswered   
4       San Jose            CA  95113-2288            NaN  ...  Unanswered   

       Veteran  NonProfit  zip_code_5    Minority  YearApproved NAICS_6  \
0   Unanswered        NaN       95125  Unanswered          2020  811121   
1  Non-Veteran        NaN       94510          No          2021  722330   
2  Non-Veteran        NaN       94107          No          2021  541511   
3   Unanswered        NaN       94551  Unanswered          2021  541420   
4  Non-Veteran        NaN       95113  Unanswered          2021  541110   

                                            LongName  \
0  Automotive Body, Paint, and Interior Repair an...   
1                               Mobile Food Services   
2              Custom Computer Programming Services    
3                         Industrial Design Services   
4                                 Offices of Lawyers   

                              IndustrySubsector NAICS_4  
0             Automotive Repair and Maintenance    8111  
1                         Special Food Services    7223  
2  Computer Systems Design and Related Services    5415  
3                   Specialized Design Services    5414  
4                                Legal Services    5411  

[5 rows x 58 columns]

### Extract Bay Area, Minority-owned businesses

Filter for businesses are minority-owned and save to file.

In [88]:
def extract_bay_minority_bus(df):
    BA_minority_PPP = df[df['Minority'] == 'Yes'].reset_index(drop=True)
    BA_minority_PPP.to_csv(d + '/ppp-loan-data/out/bay_bus_from_ppp_minority.csv', index=False)
    return BA_minority_PPP

PPP_combined_minority = extract_bay_minority_bus(PPP_combined)
PPP_combined_minority.head()

LoanNumber DateApproved  SBAOfficeCode ProcessingMethod  \
0  5224738402   02/08/2021          912.0              PPS   
1  5359188509   02/27/2021          912.0              PPS   
2  5554978603   03/20/2021          912.0              PPS   
3  9634738305   01/31/2021          912.0              PPS   
4  5004708402   02/07/2021          912.0              PPS   

              BorrowerName  BorrowerAddress   BorrowerCity BorrowerState  \
0                   ICHINA     70 Valley Ct       Atherton            CA   
1      BANZAI IZAKAYA INC.  1633 Bonanza St   Walnut Creek            CA   
2  RANGOON SUPER STARS LLC    542 Freya Way  Pleasant Hill            CA   
3         JBR PARTNERS INC   1333 Evans Ave  San Francisco            CA   
4                OSAKE INC    2446 Patio Ct     Santa Rosa            CA   

  BorrowerZip LoanStatusDate  ...        Gender      Veteran  NonProfit  \
0  94027-6472            NaN  ...  Female Owned  Non-Veteran        NaN   
1  94596-4525            NaN  ...    Male Owned  Non-Veteran        NaN   
2  94523-1712            NaN  ...    Male Owned  Non-Veteran        NaN   
3  94124-1705            NaN  ...    Male Owned  Non-Veteran        NaN   
4  95405-6737            NaN  ...  Female Owned  Non-Veteran        NaN   

   zip_code_5  Minority  YearApproved NAICS_6                   LongName  \
0       94027       Yes          2021  722511  Full-Service Restaurants    
1       94596       Yes          2021  722511  Full-Service Restaurants    
2       94523       Yes          2021  722511  Full-Service Restaurants    
3       94124       Yes          2021  541820  Public Relations Agencies   
4       95405       Yes          2021  722511  Full-Service Restaurants    

                                   IndustrySubsector NAICS_4  
0                Restaurants and Other Eating Places    7225  
1                Restaurants and Other Eating Places    7225  
2                Restaurants and Other Eating Places    7225  
3  Advertising, Public Relations, and Related Ser...    5418  
4                Restaurants and Other Eating Places    7225  

[5 rows x 58 columns]

## Part 4. Summary Data Tables for Bay Area Businesses

*Same code as Laura's below, but applied to only Bay Area businesses.*

In [72]:
PPP_ind_4 = pd.DataFrame(PPP_combined.groupby(['NAICS_4','IndustrySubsector'])['JobsReported']
                          .agg(Businesses='count',Jobs='sum').sort_values(by='Businesses', ascending=False))
PPP_ind_4.reset_index(inplace=True)
PPP_ind_4.index = np.arange(1, len(PPP_ind_4) + 1)
PPP_ind_4.head(20)

NAICS_4                                  IndustrySubsector  Businesses  \
1     7225                Restaurants and Other Eating Places       16465   
2     8121                            Personal Care Services        12794   
3     5416  Management, Scientific, and Technical Consulti...        8657   
4     4853                         Taxi and Limousine Service        8431   
5     6212                                Offices of Dentists        7328   
6     2361                  Residential Building Construction        6781   
7     6213              Offices of Other Health Practitioners        6433   
8     5419  Other Professional, Scientific, and Technical ...        6241   
9     5312          Offices of Real Estate Agents and Brokers        6215   
10    5411                                     Legal Services        5795   
11    5415       Computer Systems Design and Related Services        5187   
12    8129                           Other Personal Services         5026   
13    6211                              Offices of Physicians        4704   
14    8111                  Automotive Repair and Maintenance        4280   
15    5412  Accounting, Tax Preparation, Bookkeeping, and ...        4211   
16    4539               Other Miscellaneous Store Retailers         4040   
17    5413   Architectural, Engineering, and Related Services        4038   
18    5617                Services to Buildings and Dwellings        3619   
19    2382                     Building Equipment Contractors        3323   
20    4841                           General Freight Trucking        3195   

        Jobs  
1   338766.0  
2    43435.0  
3    48825.0  
4    11189.0  
5    51275.0  
6    48889.0  
7    28485.0  
8    35681.0  
9    15271.0  
10   30198.0  
11   59254.0  
12   24005.0  
13   34953.0  
14   33628.0  
15   19744.0  
16   30074.0  
17   43976.0  
18   36197.0  
19   50068.0  
20   13756.0

In [75]:
retail_4 = ['4411', '4412', '4413', '4421', '4422', '4431', '4441', '4442', '4451', '4452', '4453', '4461', '4471', '4481', 
            '4482', '4483', '4511', '4512', '4522', '4523', '4531', '4532', '4533', '4539', '4541', '4542', '4543']

In [76]:
PPP_combined['NAICS_4'].dtypes

dtype('O')

In [77]:
PPP_retail_4 = PPP_combined[PPP_combined['NAICS_4'].isin(retail_4)]
PPP_retail_4

LoanNumber DateApproved  SBAOfficeCode ProcessingMethod  \
47     4819837700   05/01/2020          912.0              PPP   
99     3317428305   01/21/2021          912.0              PPS   
169    5462957005   04/05/2020          912.0              PPP   
195    7119297108   04/14/2020          912.0              PPP   
208    6293057706   05/01/2020          912.0              PPP   
...           ...          ...            ...              ...   
33129  8371687408   05/18/2020          912.0              PPP   
33135  8728438403   02/13/2021          912.0              PPS   
33143  9079687002   04/09/2020          912.0              PPP   
33168  9958578508   03/12/2021          912.0              PPS   
33169  9989087303   05/03/2020          912.0              PPP   

                          BorrowerName           BorrowerAddress  \
47                   SAM'S ELECTRO INC      1555 BOTELHO DR. 437   
99                   PCIFIC FUELS INC.        46494 Mission Blvd   
169        TIN RX CASTRO SAN FRANCISCO        2175 Market Street   
195                  EZRA CONSTRUCTION        1156 Keeler Avenue   
208    STRATEGIC BUILDING PRODUCTS LLC            10920 BIGGE ST   
...                                ...                       ...   
33129      NAPA BARREL REPAIR SHOP INC  918 ENTERPRISE WAY STE K   
33135      U.S. HISPANIC VENTURES INC.           2400 Mission St   
33143                     GOTFLOOR.COM      2240 DE LA CRUZ BLVD   
33168    CANADIAN-AMERICAN OIL COMPANY   444 Divisadero St # 100   
33169  SANTA ROSA SEAFOOD RETAIL, INC.        946 Santa Rosa Ave   

        BorrowerCity BorrowerState BorrowerZip LoanStatusDate  ...  \
47      WALNUT CREEK            CA       94596            NaN  ...   
99           Fremont            CA  94539-7063            NaN  ...   
169    SAN FRANCISCO            CA  94114-1321     02/24/2021  ...   
195         Berkeley            CA       94708            NaN  ...   
208      SAN LEANDRO            CA  94577-1121     03/18/2021  ...   
...              ...           ...         ...            ...  ...   
33129           NAPA            CA  94558-6230            NaN  ...   
33135  San Francisco            CA  94110-2415            NaN  ...   
33143    SANTA CLARA            CA  95050-3008     04/16/2021  ...   
33168  San Francisco            CA  94117-2211            NaN  ...   
33169     SANTA ROSA            CA       95404     04/23/2021  ...   

             Gender      Veteran  NonProfit  zip_code_5    Minority  \
47       Unanswered   Unanswered        NaN       94596  Unanswered   
99       Male Owned  Non-Veteran        NaN       94539          No   
169    Female Owned  Non-Veteran        NaN       94114          No   
195      Unanswered   Unanswered        NaN       94708  Unanswered   
208      Unanswered   Unanswered        NaN       94577  Unanswered   
...             ...          ...        ...         ...         ...   
33129    Unanswered   Unanswered        NaN       94558  Unanswered   
33135    Male Owned  Non-Veteran        NaN       94110          No   
33143    Male Owned  Non-Veteran        NaN       95050  Unanswered   
33168    Unanswered   Unanswered        NaN       94117  Unanswered   
33169    Unanswered   Unanswered        NaN       95404  Unanswered   

       YearApproved NAICS_6  \
47             2020  453998   
99             2021  447110   
169            2020  446110   
195            2020  453998   
208            2020  444110   
...             ...     ...   
33129          2020  443141   
33135          2021  446191   
33143          2020  444190   
33168          2021  447110   
33169          2020  445220   

                                                LongName  \
47     All Other Miscellaneous Store Retailers (excep...   
99            Gasoline Stations with Convenience Stores    
169                          Pharmacies and Drug Stores    
195    All Other Miscellaneous Store Retailers (excep...   
208                           

In [78]:
personal_4 = ['8121', '8122', '8123', '8129']

In [79]:
PPP_personal_4 = PPP_combined[PPP_combined['NAICS_4'].isin(personal_4)]
PPP_personal_4

LoanNumber DateApproved  SBAOfficeCode ProcessingMethod  \
78     3863827700   05/01/2020          912.0              PPP   
79     7010998305   01/27/2021          912.0              PPS   
122    8598858303   01/29/2021          912.0              PPS   
123    2147418108   07/11/2020          912.0              PPP   
167    5093848301   01/25/2021          912.0              PPS   
...           ...          ...            ...              ...   
32977  3997997809   05/27/2020          912.0              PPP   
33083  6975237208   04/28/2020          912.0              PPP   
33110  7858308403   02/12/2021          912.0              PPS   
33112  7915248504   03/08/2021          912.0              PPS   
33145  9120708506   03/12/2021          912.0              PPS   

                         BorrowerName          BorrowerAddress   BorrowerCity  \
78                KLUB K9 PLAY CENTER        174 COMMERCIAL ST      SUNNYVALE   
79            KLUB K9 PLAY CENTER INC        174 Commercial St      Sunnyvale   
122                        SOFTAP INC          1046 Country Ln     Pleasanton   
123                   JACQUELINES LLC  3960 Adeline Street 108     Emeryville   
167                        SPROOS INC      552 San Anselmo Ave    San Anselmo   
...                               ...                      ...            ...   
32977  J'S BARBER SHOP AND HAIR SALON        141 Sunset Avenue    Suisun City   
33083                   MADUSALON INC    300 Divisadero Street  San Francisco   
33110             ARCHIMEDES BANYA SF            748 Innes Ave  San Francisco   
33112                 SEVEN SALON INC         5358 College Ave        Oakland   
33145        A TOUCH OF ELEGANCE INC.            1150 Crane St     Menlo Park   

      BorrowerState BorrowerZip LoanStatusDate  ...        Gender  \
78               CA       94086            NaN  ...    Unanswered   
79               CA  94086-5201            NaN  ...    Unanswered   
122              CA  94588-9515            NaN  ...    Unanswered   
123              CA  94608-3511            NaN  ...    Unanswered   
167              CA  94960-2621            NaN  ...    Unanswered   
...             ...         ...            ...  ...           ...   
32977            CA  94585-2063            NaN  ...    Unanswered   
33083            CA  94117-2209            NaN  ...    Unanswered   
33110            CA  94124-3054            NaN  ...    Unanswered   
33112            CA  94618-1417            NaN  ...  Female Owned   
33145            CA  94025-4314            NaN  ...  Female Owned   

           Veteran  NonProfit  zip_code_5    Minority  YearApproved NAICS_6  \
78      Unanswered        NaN       94086  Unanswered          2020  812910   
79      Unanswered        NaN       94086  Unanswered          2021  812910   
122     Unanswered        NaN       94588  Unanswered          2021  812199   
123     Unanswered        NaN       94608  Unanswered          2020  812990   
167     Unanswered        NaN       94960  Unanswered          2021  812112   
...            ...        ...         ...         ...           ...     ...   
32977   Unanswered        NaN       94585  Unanswered          2020  812990   
33083   Unanswered        NaN       94117  Unanswered          2020  812112   
33110   Unanswered        NaN       94124  Unanswered          2021  812199   
33112  Non-Veteran        NaN       94618         Yes          2021  812112   
33145  Non-Veteran        NaN       94025         Yes          2021  812112   

                                     LongName         IndustrySubsector  \
78     Pet Care (except Veterinary) Services   Other Personal Services    
79     Pet Care (except Veterinary) Services   Other Personal Services    
122             Other Personal Care Services    Personal Care Services    
123              All Other Personal Services   Other Personal Services    
167                            Beauty Salons    Personal Care Services    
...          

In [80]:
food_4 = ['7223', '7224', '7225']

In [81]:
PPP_food_4 = PPP_combined[PPP_combined['NAICS_4'].isin(food_4)]
PPP_food_4

LoanNumber DateApproved  SBAOfficeCode ProcessingMethod  \
1      1464468601   03/13/2021          912.0              PPS   
6      5224738402   02/08/2021          912.0              PPS   
7      5359188509   02/27/2021          912.0              PPS   
8      5422668907   04/30/2021          912.0              PPS   
9      5554978603   03/20/2021          912.0              PPS   
...           ...          ...            ...              ...   
33140  8962538407   02/14/2021          912.0              PPS   
33150  9360078408   02/16/2021          912.0              PPS   
33151  9403758303   01/30/2021          912.0              PPS   
33152  9479807109   04/15/2020          912.0              PPP   
33166  9940228401   02/18/2021          912.0              PPS   

                           BorrowerName       BorrowerAddress   BorrowerCity  \
1             GOLD RUSH KETTLE KORN LLC   4690 E 2nd St Ste 9        Benicia   
6                                ICHINA          70 Valley Ct       Atherton   
7                   BANZAI IZAKAYA INC.       1633 Bonanza St   Walnut Creek   
8                       RIVA CUCINA LLC  800 Heinz Ave Ste 19       Berkeley   
9               RANGOON SUPER STARS LLC         542 Freya Way  Pleasant Hill   
...                                 ...                   ...            ...   
33140       PYEONG CHANG TOFU HOUSE INC    4701 Telegraph Ave        Oakland   
33150                   SIRI GROUP INC.        1175 Folsom St  San Francisco   
33151                            AKINAI           2092 3rd St  San Francisco   
33152  UNIVERSAL ENTERPRISE (USA), INC.          76 S ABEL ST       MILPITAS   
33166               A.L. COMPANEROS INC         101 E Napa St         Sonoma   

      BorrowerState BorrowerZip LoanStatusDate  ...        Gender  \
1                CA  94510-1008            NaN  ...    Male Owned   
6                CA  94027-6472            NaN  ...  Female Owned   
7                CA  94596-4525            NaN  ...    Male Owned   
8                CA  94710-2747     04/30/2021  ...  Female Owned   
9                CA  94523-1712            NaN  ...    Male Owned   
...             ...         ...            ...  ...           ...   
33140            CA  94609-2023            NaN  ...    Unanswered   
33150            CA  94103-3930            NaN  ...  Female Owned   
33151            CA  94107-3122            NaN  ...    Male Owned   
33152            CA  95035-5251     04/16/2021  ...  Female Owned   
33166            CA  95476-6709            NaN  ...    Unanswered   

           Veteran  NonProfit  zip_code_5    Minority  YearApproved NAICS_6  \
1      Non-Veteran        NaN       94510          No          2021  722330   
6      Non-Veteran        NaN       94027         Yes          2021  722511   
7      Non-Veteran        NaN       94596         Yes          2021  722511   
8      Non-Veteran        NaN       94710          No          2021  722511   
9      Non-Veteran        NaN       94523         Yes          2021  722511   
...            ...        ...         ...         ...           ...     ...   
33140   Unanswered        NaN       94609  Unanswered          2021  722511   
33150  Non-Veteran        NaN       94103         Yes          2021  722511   
33151      Veteran        NaN       94107         Yes          2021  722511   
33152  Non-Veteran        NaN       95035          No          2020  722511   
33166   Unanswered        NaN       95476  Unanswered          2021  722511   

                        LongName                    IndustrySubsector NAICS_4  
1           Mobile Food Services                Special Food Services    7223  
6      Full-Service Restaurants   Restaurants and Other Eating Places    7225  
7      Full-Service Restaurants   Restaurants and Other Eating Places    7225  
8      Full-Service Restaurants   Restaurants and Other Eating Places    7225  
9      Full-Service Restaurants   Restaurants and Other Eating Places    72

In [97]:
PPP_minority_ind_4 = pd.DataFrame(PPP_combined_minority.groupby(['NAICS_4','IndustrySubsector'])['JobsReported']
                          .agg(Businesses='count',Jobs='sum').sort_values(by='Businesses', ascending=False))
PPP_minority_ind_4.reset_index(inplace=True)
PPP_minority_ind_4.index = np.arange(1, len(PPP_minority_ind_4) + 1)
PPP_minority_ind_4.head(20)

NAICS_4                                  IndustrySubsector  Businesses  \
1     7225                Restaurants and Other Eating Places        3272   
2     8121                            Personal Care Services         2445   
3     6212                                Offices of Dentists        1361   
4     4853                         Taxi and Limousine Service        1023   
5     4841                           General Freight Trucking         927   
6     8129                           Other Personal Services          904   
7     5416  Management, Scientific, and Technical Consulti...         793   
8     2361                  Residential Building Construction         720   
9     5419  Other Professional, Scientific, and Technical ...         652   
10    6211                              Offices of Physicians         640   
11    7211                             Traveler Accommodation         629   
12    5312          Offices of Real Estate Agents and Brokers         626   
13    8111                  Automotive Repair and Maintenance         614   
14    6213              Offices of Other Health Practitioners         600   
15    5415       Computer Systems Design and Related Services         597   
16    6244                            Child Day Care Services         588   
17    5617                Services to Buildings and Dwellings         517   
18    4539               Other Miscellaneous Store Retailers          485   
19    5412  Accounting, Tax Preparation, Bookkeeping, and ...         464   
20    5242  Agencies, Brokerages, and Other Insurance Rela...         424   

       Jobs  
1   58598.0  
2    8615.0  
3   10006.0  
4    1264.0  
5    2392.0  
6    3985.0  
7    4012.0  
8    2887.0  
9    3356.0  
10   3273.0  
11   7095.0  
12   1234.0  
13   4746.0  
14   2973.0  
15   6326.0  
16   4188.0  
17   4090.0  
18   2368.0  
19   2089.0  
20    997.0

In [98]:
PPP_minority_retail_4 = PPP_minority_ind_4[PPP_minority_ind_4['NAICS_4'].isin(retail_4)]
PPP_minority_retail_4

NAICS_4                                  IndustrySubsector  Businesses  \
18     4539               Other Miscellaneous Store Retailers          485   
25     4481                                   Clothing Stores          327   
30     4471                                 Gasoline Stations          231   
32     4461                   Health and Personal Care Stores          223   
35     4451                                    Grocery Stores          199   
44     4452                             Specialty Food Stores          137   
67     4453                     Beer, Wine, and Liquor Stores           79   
69     4483        Jewelry, Luggage, and Leather Goods Stores           74   
79     4541         Electronic Shopping and Mail-Order Houses           64   
87     4511  Sporting Goods, Hobby, and Musical Instrument ...          55   
88     4532      Office Supplies, Stationery, and Gift Stores           54   
90     4411                                Automobile Dealers           52   
99     4441            Building Material and Supplies Dealers           48   
101    4413    Automotive Parts, Accessories, and Tire Stores           43   
102    4422                           Home Furnishings Stores           43   
104    4431                  Electronics and Appliance Stores           41   
105    4543                     Direct Selling Establishments           41   
106    4531                                          Florists           40   
111    4421                                  Furniture Stores           36   
114    4523  General Merchandise Stores, including Warehous...          33   
152    4482                                       Shoe Stores           15   
179    4442     Lawn and Garden Equipment and Supplies Stores            9   
181    4412                       Other Motor Vehicle Dealers            8   
190    4533                           Used Merchandise Stores            7   
192    4542                         Vending Machine Operators            7   
207    4512                      Book Stores and News Dealers            6   
232    4522                                 Department Stores            4   

       Jobs  
18   2368.0  
25   1303.0  
30   2497.0  
32   1003.0  
35   3319.0  
44   1395.0  
67    806.0  
69    322.0  
79    243.0  
87    290.0  
88    141.0  
90    623.0  
99    480.0  
101   285.0  
102   222.0  
104   170.0  
105   144.0  
106   141.0  
111   106.0  
114   137.0  
152    96.0  
179   130.0  
181    57.0  
190    19.0  
192    10.0  
207   355.0  
232    15.0

In [99]:
PPP_combined['YearApproved'].unique()

array(['2020', '2021'], dtype=object)

In [100]:
#Extract for 2021 approvals only 
PPP_combined_2021 = PPP_combined[PPP_combined['YearApproved']=='2021']
PPP_combined_2021.head(10)

LoanNumber DateApproved  SBAOfficeCode ProcessingMethod  \
1   1464468601   03/13/2021          912.0              PPS   
2   2663308509   02/22/2021          912.0              PPS   
3   4137348607   03/18/2021          912.0              PPS   
4   4315748602   03/18/2021          912.0              PPS   
5   4337488902   04/28/2021          912.0              PPS   
6   5224738402   02/08/2021          912.0              PPS   
7   5359188509   02/27/2021          912.0              PPS   
8   5422668907   04/30/2021          912.0              PPS   
9   5554978603   03/20/2021          912.0              PPS   
10  5743508503   03/01/2021          912.0              PPS   

                    BorrowerName            BorrowerAddress   BorrowerCity  \
1      GOLD RUSH KETTLE KORN LLC        4690 E 2nd St Ste 9        Benicia   
2               KALIBER LABS INC       188 King St Unit 307  San Francisco   
3      ROSS MCDONALD COMPANY INC            1154 Stealth St      Livermore   
4         GATES EISENHART DAWSON   125 S Market St Ste 1200       San Jose   
5   INNOVATIVE CONTROL SOLUTIONS           383 Princeton Ln       Danville   
6                         ICHINA               70 Valley Ct       Atherton   
7            BANZAI IZAKAYA INC.            1633 Bonanza St   Walnut Creek   
8                RIVA CUCINA LLC       800 Heinz Ave Ste 19       Berkeley   
9        RANGOON SUPER STARS LLC              542 Freya Way  Pleasant Hill   
10    SORELLE ITALIAN BISTRO LLC  915 S San Tomas Aquino Rd       Campbell   

   BorrowerState BorrowerZip LoanStatusDate  ...        Gender      Veteran  \
1             CA  94510-1008            NaN  ...    Male Owned  Non-Veteran   
2             CA  94107-4903            NaN  ...    Male Owned  Non-Veteran   
3             CA  94551-9300            NaN  ...    Unanswered   Unanswered   
4             CA  95113-2288            NaN  ...    Unanswered  Non-Veteran   
5             CA  94526-4125     04/28/2021  ...    Male Owned  Non-Veteran   
6             CA  94027-6472            NaN  ...  Female Owned  Non-Veteran   
7             CA  94596-4525            NaN  ...    Male Owned  Non-Veteran   
8             CA  94710-2747     04/30/2021  ...  Female Owned  Non-Veteran   
9             CA  94523-1712            NaN  ...    Male Owned  Non-Veteran   
10            CA  95008-4449            NaN  ...    Unanswered   Unanswered   

    NonProfit  zip_code_5    Minority  YearApproved NAICS_6  \
1         NaN       94510          No          2021  722330   
2         NaN       94107          No          2021  541511   
3         NaN       94551  Unanswered          2021  541420   
4         NaN       95113  Unanswered          2021  541110   
5         NaN       94526  Unanswered          2021  238210   
6         NaN       94027         Yes          2021  722511   
7         NaN       94596         Yes          2021  722511   
8         NaN       94710          No          2021  722511   
9         NaN       94523         Yes          2021  722511   
10        NaN       95008  Unanswered          2021  722511   

                                             LongName  \
1                                Mobile Food Services   
2               Custom Computer Programming Services    
3                          Industrial Design Services   
4                                  Offices of Lawyers   
5   Electrical Contractors and Other Wiring Instal...   
6                           Full-Service Restaurants    
7                           Full-Service Restaurants    
8                           Full-Service Restaurants    
9                           Full-Service Restaurants    
10                          Full-Service Restaurants    

                               IndustrySubsector NAICS_4  
1                          Special Food Services    7223  
2   Computer Systems Design and Related Services    5415  
3                    Specialized Design Services    5414  
4                      

In [101]:
PPP_ind_4_2021 = pd.DataFrame(PPP_combined_2021.groupby(['NAICS_4','IndustrySubsector'])['JobsReported']
                          .agg(Businesses='count',Jobs='sum').sort_values(by='Businesses', ascending=False))
PPP_ind_4_2021.reset_index(inplace=True)
PPP_ind_4_2021.index = np.arange(1, len(PPP_ind_4_2021) + 1)
PPP_ind_4_2021.head(20)

NAICS_4                                  IndustrySubsector  Businesses  \
1     8121                            Personal Care Services         8098   
2     7225                Restaurants and Other Eating Places        8017   
3     4853                         Taxi and Limousine Service        7224   
4     5416  Management, Scientific, and Technical Consulti...        3645   
5     6212                                Offices of Dentists        3287   
6     2361                  Residential Building Construction        3048   
7     5312          Offices of Real Estate Agents and Brokers        2928   
8     6213              Offices of Other Health Practitioners        2720   
9     5419  Other Professional, Scientific, and Technical ...        2706   
10    8129                           Other Personal Services         2700   
11    5411                                     Legal Services        2248   
12    4841                           General Freight Trucking        2001   
13    5415       Computer Systems Design and Related Services        1967   
14    5617                Services to Buildings and Dwellings        1886   
15    8111                  Automotive Repair and Maintenance        1873   
16    7223                              Special Food Services        1835   
17    5412  Accounting, Tax Preparation, Bookkeeping, and ...        1773   
18    6244                            Child Day Care Services        1709   
19    6211                              Offices of Physicians        1688   
20    4921             Couriers and Express Delivery Services        1569   

        Jobs  
1    20894.0  
2   140364.0  
3     8317.0  
4    12465.0  
5    23031.0  
6    16650.0  
7     4708.0  
8    10629.0  
9     8718.0  
10    9445.0  
11    8028.0  
12    4836.0  
13   13784.0  
14   10914.0  
15   11618.0  
16   10503.0  
17    5345.0  
18   10278.0  
19   10583.0  
20    2632.0

In [93]:
PPP_retail_4_2021 = PPP_ind_4_2021[PPP_ind_4_2021['NAICS_4'].isin(retail_4)]
PPP_retail_4_2021

NAICS_4                                  IndustrySubsector  Businesses  \
23     4539               Other Miscellaneous Store Retailers         1458   
32     4481                                   Clothing Stores         1013   
41     4461                   Health and Personal Care Stores          636   
51     4471                                 Gasoline Stations          483   
65     4451                                    Grocery Stores          328   
71     4483        Jewelry, Luggage, and Leather Goods Stores          304   
76     4452                             Specialty Food Stores          257   
77     4411                                Automobile Dealers          252   
78     4511  Sporting Goods, Hobby, and Musical Instrument ...         251   
81     4541         Electronic Shopping and Mail-Order Houses          249   
87     4543                     Direct Selling Establishments          191   
88     4532      Office Supplies, Stationery, and Gift Stores          188   
93     4422                           Home Furnishings Stores          179   
99     4531                                          Florists          172   
102    4421                                  Furniture Stores          162   
111    4413    Automotive Parts, Accessories, and Tire Stores          139   
112    4441            Building Material and Supplies Dealers          134   
116    4431                  Electronics and Appliance Stores          131   
120    4523  General Merchandise Stores, including Warehous...         125   
121    4453                     Beer, Wine, and Liquor Stores          124   
145    4533                           Used Merchandise Stores           75   
156    4482                                       Shoe Stores           60   
160    4512                      Book Stores and News Dealers           58   
164    4542                         Vending Machine Operators           52   
178    4412                       Other Motor Vehicle Dealers           44   
195    4442     Lawn and Garden Equipment and Supplies Stores           32   
273    4522                                 Department Stores            5   

       Jobs  
23   5436.0  
32   3792.0  
41   1999.0  
51   6091.0  
65   2810.0  
71   1325.0  
76   1903.0  
77   3978.0  
78   1418.0  
81   1101.0  
87    425.0  
88   1247.0  
93   1182.0  
99    541.0  
102  1284.0  
111   809.0  
112  1333.0  
116   532.0  
120   536.0  
121   529.0  
145   852.0  
156   444.0  
160   524.0  
164   321.0  
178   249.0  
195   224.0  
273    16.0

In [94]:
PPP_combined_minority_2021 = PPP_combined_2021[PPP_combined_2021['Minority']=='Yes']
PPP_combined_minority_2021.head(10)

LoanNumber DateApproved  SBAOfficeCode ProcessingMethod  \
6   5224738402   02/08/2021          912.0              PPS   
7   5359188509   02/27/2021          912.0              PPS   
9   5554978603   03/20/2021          912.0              PPS   
17  9634738305   01/31/2021          912.0              PPS   
18  5004708402   02/07/2021          912.0              PPS   
42  5147588302   01/25/2021          912.0              PPS   
46  3404988405   02/04/2021          912.0              PPS   
50  6070528704   04/03/2021          912.0              PPS   
55  6408068401   02/10/2021          912.0              PPS   
57  2347158510   02/20/2021          912.0              PPS   

               BorrowerName         BorrowerAddress   BorrowerCity  \
6                    ICHINA            70 Valley Ct       Atherton   
7       BANZAI IZAKAYA INC.         1633 Bonanza St   Walnut Creek   
9   RANGOON SUPER STARS LLC           542 Freya Way  Pleasant Hill   
17         JBR PARTNERS INC          1333 Evans Ave  San Francisco   
18                OSAKE INC           2446 Patio Ct     Santa Rosa   
42       KVS PRODUCTION INC  601 Mission Bay Blvd N  San Francisco   
46      RED CHILI GROUP INC      29583 Mission Blvd        Hayward   
50              SPARX 7 LLC         333 Potrero Ave  San Francisco   
55       PRESTIGE FOODS INC       15900 Monterey St    Morgan Hill   
57      BLUE SKY FOODS INC.      2300 Polvorosa Ave    San Leandro   

   BorrowerState BorrowerZip LoanStatusDate  ...        Gender      Veteran  \
6             CA  94027-6472            NaN  ...  Female Owned  Non-Veteran   
7             CA  94596-4525            NaN  ...    Male Owned  Non-Veteran   
9             CA  94523-1712            NaN  ...    Male Owned  Non-Veteran   
17            CA  94124-1705            NaN  ...    Male Owned  Non-Veteran   
18            CA  95405-6737            NaN  ...  Female Owned  Non-Veteran   
42            CA  94158-2472            NaN  ...    Male Owned      Veteran   
46            CA  94544-6129            NaN  ...    Male Owned   Unanswered   
50            CA  94103-4816     04/03/2021  ...    Male Owned   Unanswered   
55            CA  95037-5444            NaN  ...  Female Owned  Non-Veteran   
57            CA  94577-2218            NaN  ...  Female Owned  Non-Veteran   

    NonProfit  zip_code_5  Minority  YearApproved NAICS_6  \
6         NaN       94027       Yes          2021  722511   
7         NaN       94596       Yes          2021  722511   
9         NaN       94523       Yes          2021  722511   
17        NaN       94124       Yes          2021  541820   
18        NaN       95405       Yes          2021  722511   
42        NaN       94158       Yes          2021  722513   
46        NaN       94544       Yes          2021  722511   
50        NaN       94103       Yes          2021  721310   
55        NaN       95037       Yes          2021  722511   
57        NaN       94577       Yes          2021  424490   

                                             LongName  \
6                           Full-Service Restaurants    
7                           Full-Service Restaurants    
9                           Full-Service Restaurants    
17                          Public Relations Agencies   
18                          Full-Service Restaurants    
42                       Limited-Service Restaurants    
46                          Full-Service Restaurants    
50  Rooming and Boarding Houses, Dormitories, and ...   
55                          Full-Service Restaurants    
57  Other Grocery and Related Products Merchant Wh...   

                                    IndustrySubsector NAICS_4  
6                 Restaurants and Other Eating Places    7225  
7                 Restaurants and Other Eating Places    7225  
9                 Restaurants and Other Eating Places    7225  
17  Advertising, Public Relations, and Related Ser...    5418  
18                Restaurants and Other Eating Pl

In [95]:
PPP_minority_ind_4_2021 = pd.DataFrame(PPP_combined_minority_2021.groupby(['NAICS_4','IndustrySubsector'])['JobsReported']
                          .agg(Businesses='count',Jobs='sum').sort_values(by='Businesses', ascending=False))
PPP_minority_ind_4_2021.reset_index(inplace=True)
PPP_minority_ind_4_2021.index = np.arange(1, len(PPP_minority_ind_4_2021) + 1)
PPP_minority_ind_4_2021.head(20)

NAICS_4                                  IndustrySubsector  Businesses  \
1     8121                            Personal Care Services         1853   
2     7225                Restaurants and Other Eating Places        1850   
3     4853                         Taxi and Limousine Service         890   
4     4841                           General Freight Trucking         732   
5     6212                                Offices of Dentists         708   
6     8129                           Other Personal Services          631   
7     2361                  Residential Building Construction         504   
8     5416  Management, Scientific, and Technical Consulti...         463   
9     5312          Offices of Real Estate Agents and Brokers         410   
10    7211                             Traveler Accommodation         402   
11    6244                            Child Day Care Services         389   
12    5419  Other Professional, Scientific, and Technical ...         380   
13    5617                Services to Buildings and Dwellings         373   
14    8111                  Automotive Repair and Maintenance         363   
15    5415       Computer Systems Design and Related Services         361   
16    6211                              Offices of Physicians         349   
17    6213              Offices of Other Health Practitioners         341   
18    7223                              Special Food Services         305   
19    5242  Agencies, Brokerages, and Other Insurance Rela...         295   
20    4539               Other Miscellaneous Store Retailers          294   

       Jobs  
1    4499.0  
2   26413.0  
3    1000.0  
4    1456.0  
5    4916.0  
6    2147.0  
7    1608.0  
8    1328.0  
9     691.0  
10   3945.0  
11   2096.0  
12   1281.0  
13   1605.0  
14   1742.0  
15   2798.0  
16   1542.0  
17   1540.0  
18   1010.0  
19    539.0  
20    798.0

In [96]:
PPP_minority_retail_4_2021 = PPP_minority_ind_4_2021[PPP_minority_ind_4_2021['NAICS_4'].isin(retail_4)]
PPP_minority_retail_4_2021

NAICS_4                                  IndustrySubsector  Businesses  \
20     4539               Other Miscellaneous Store Retailers          294   
23     4481                                   Clothing Stores          254   
28     4461                   Health and Personal Care Stores          152   
35     4471                                 Gasoline Stations          126   
51     4451                                    Grocery Stores           66   
59     4541         Electronic Shopping and Mail-Order Houses           52   
66     4452                             Specialty Food Stores           46   
68     4483        Jewelry, Luggage, and Leather Goods Stores           44   
77     4532      Office Supplies, Stationery, and Gift Stores           35   
83     4411                                Automobile Dealers           33   
86     4511  Sporting Goods, Hobby, and Musical Instrument ...          29   
89     4441            Building Material and Supplies Dealers           28   
92     4543                     Direct Selling Establishments           27   
95     4431                  Electronics and Appliance Stores           26   
98     4413    Automotive Parts, Accessories, and Tire Stores           25   
99     4453                     Beer, Wine, and Liquor Stores           25   
104    4531                                          Florists           23   
105    4422                           Home Furnishings Stores           23   
108    4523  General Merchandise Stores, including Warehous...          20   
109    4421                                  Furniture Stores           20   
145    4482                                       Shoe Stores           10   
171    4442     Lawn and Garden Equipment and Supplies Stores            6   
194    4512                      Book Stores and News Dealers            3   
195    4522                                 Department Stores            3   
204    4412                       Other Motor Vehicle Dealers            3   
205    4533                           Used Merchandise Stores            3   
206    4542                         Vending Machine Operators            3   

       Jobs  
20    798.0  
23    674.0  
28    460.0  
35   1318.0  
51    497.0  
59    137.0  
66    286.0  
68    176.0  
77     80.0  
83    267.0  
86    150.0  
89    198.0  
92     31.0  
95     94.0  
98    116.0  
99     87.0  
104    93.0  
105    98.0  
108    81.0  
109    62.0  
145    41.0  
171    66.0  
194     3.0  
195    14.0  
204    17.0  
205     9.0  
206     3.0

## Part 5. PPP Loan Data (Unfiltered - No Zip Code Analysis)

By Laura Schmahmann

Note: Heavy on PC memory, couldn't run on my end, see above for filtered version.

In [3]:
#Import Paycheck Protection Program Loan Information
ppp_sub_150_1 = pd.read_csv('public_up_to_150k_1.csv')
ppp_sub_150_2 = pd.read_csv('public_up_to_150k_2.csv')
ppp_sub_150_3 = pd.read_csv('public_up_to_150k_3.csv')
ppp_sub_150_4 = pd.read_csv('public_up_to_150k_4.csv')
ppp_sub_150_5 = pd.read_csv('public_up_to_150k_5.csv')
ppp_sub_150_6 = pd.read_csv('public_up_to_150k_6.csv')
ppp_sub_150_7 = pd.read_csv('public_up_to_150k_7.csv')
ppp_sub_150_8 = pd.read_csv('public_up_to_150k_8.csv')
ppp_sub_150_9 = pd.read_csv('public_up_to_150k_9.csv')
ppp_sub_150_10 = pd.read_csv('public_up_to_150k_10.csv')
ppp_sub_150_11 = pd.read_csv('public_up_to_150k_11.csv')
ppp_plus_150 = pd.read_csv('public_150k_plus.csv')


C:\Users\Trang\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (16,50) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Trang\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (50) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
ppp_plus_150.head()

LoanNumber DateApproved  SBAOfficeCode ProcessingMethod  \
0  9547507704   05/01/2020            464              PPP   
1  9777677704   05/01/2020            464              PPP   
2  5791407702   05/01/2020           1013              PPP   
3  6223567700   05/01/2020            920              PPP   
4  9662437702   05/01/2020            101              PPP   

               BorrowerName        BorrowerAddress      BorrowerCity  \
0     SUMTER COATINGS, INC.  2410 Highway 15 South            Sumter   
1     PLEASANT PLACES, INC.    7684 Southrail Road  North Charleston   
2   BOYER CHILDREN'S CLINIC       1850 BOYER AVE E           SEATTLE   
3  KIRTLEY CONSTRUCTION INC   1661 MARTIN RANCH RD    SAN BERNARDINO   
4              AERO BOX LLC                    NaN               NaN   

  BorrowerState BorrowerZip LoanStatusDate  ... HEALTH_CARE_PROCEED  \
0           NaN  29150-9662     12/18/2020  ...                 NaN   
1           NaN  29420-9000            NaN  ...                 NaN   
2           NaN  98112-2922     03/17/2021  ...                 NaN   
3           NaN  92407-1740            NaN  ...                 NaN   
4           NaN         NaN            NaN  ...                 NaN   

   DEBT_INTEREST_PROCEED             BusinessType  \
0                    NaN              Corporation   
1                    NaN      Sole Proprietorship   
2                    NaN  Non-Profit Organization   
3                    NaN              Corporation   
4                    NaN                      NaN   

   OriginatingLenderLocationID                      OriginatingLender  \
0                        19248                           Synovus Bank   
1                        19248                           Synovus Bank   
2                         9551  Bank of America, National Association   
3                         9551  Bank of America, National Association   
4                        57328           The Huntington National Bank   

   OriginatingLenderCity OriginatingLenderState      Gender      Veteran  \
0               COLUMBUS                     GA  Unanswered   Unanswered   
1               COLUMBUS                     GA  Male Owned  Non-Veteran   
2              CHARLOTTE                     NC  Unanswered   Unanswered   
3              CHARLOTTE                     NC  Unanswered   Unanswered   
4               COLUMBUS                     OH  Unanswered   Unanswered   

  NonProfit  
0       NaN  
1       NaN  
2         Y  
3       NaN  
4       NaN  

[5 rows x 51 columns]

In [5]:
#Combine PPP datasets
PPP_datasets = [ppp_sub_150_1,ppp_sub_150_2,ppp_sub_150_3,ppp_sub_150_4,ppp_sub_150_5,ppp_sub_150_6,
                ppp_sub_150_7,ppp_sub_150_8,ppp_sub_150_9,ppp_sub_150_10,ppp_sub_150_11,ppp_plus_150]

In [6]:
PPP_combined = pd.concat(PPP_datasets)

In [7]:
PPP_combined.head(10)

LoanNumber DateApproved  SBAOfficeCode ProcessingMethod  \
0  5375617707   05/01/2020          101.0              PPP   
1  9677497701   05/01/2020          464.0              PPP   
2  9547167709   05/01/2020          464.0              PPP   
3  6448037706   05/01/2020          515.0              PPP   
4  9609017706   05/01/2020          464.0              PPP   
5  9789867710   05/01/2020          101.0              PPP   
6  9589997709   05/01/2020          101.0              PPP   
7  6486007709   05/01/2020          914.0              PPP   
8  9662387700   05/01/2020          101.0              PPP   
9  2767027201   04/16/2020            NaN              PPP   

                                     BorrowerName            BorrowerAddress  \
0                                   NOT AVAILABLE                        NaN   
1          NORTH CHARLESTON HOSPITALITY GROUP LLC        192 College Park Rd   
2                            Q AND J SERVICES LLC    301 Old Georgetown Road   
3  OPTIMIZED PROCESS SOLUTIONS DBA AAA INDUSTRIES              24500 CAPITOL   
4                            D2 POWER SPORTS, LLC             125 Simuel Dr.   
5                    VULCAN MACHINERY CORPORATION                        NaN   
6                TJK KITCHENS &AMP; BREWPUBS, LLC                        NaN   
7                               SEAN T KY DDS INC  200 S EL MOLINO AVE STE 5   
8                                   RON GOLDSTONE                        NaN   
9                                     Exemption 6                        NaN   

  BorrowerCity BorrowerState BorrowerZip LoanStatusDate  ...  \
0          NaN           NaN         NaN            NaN  ...   
1       Ladson           NaN  29456-3517            NaN  ...   
2      Manning           NaN  29102-2734     04/20/2021  ...   
3      REDFORD           NaN  48239-2446     04/16/2021  ...   
4  Spartanburg           NaN  29303-2085            NaN  ...   
5          NaN           NaN         NaN            NaN  ...   
6          NaN           NaN         NaN            NaN  ...   
7     PASADENA           NaN  91101-2985            NaN  ...   
8          NaN           NaN         NaN            NaN  ...   
9          NaN           NaN         NaN     02/18/2021  ...   

  HEALTH_CARE_PROCEED  DEBT_INTEREST_PROCEED                     BusinessType  \
0                 NaN                    NaN                              NaN   
1                 NaN                    NaN  Limited  Liability Company(LLC)   
2                 NaN                    NaN  Limited  Liability Company(LLC)   
3                 NaN                    NaN  Limited  Liability Company(LLC)   
4                 NaN                    NaN  Limited  Liability Company(LLC)   
5                 NaN                    NaN                      Corporation   
6                 NaN                    NaN  Limited  Liability Company(LLC)   
7                 NaN                    NaN                      Corporation   
8                 NaN                    NaN                              NaN   
9                 0.0                    0.0         Subchapter S Corporation   

   OriginatingLenderLocationID                      OriginatingLender  \
0                       9551.0  Bank of America, National Association   
1                      19248.0                           Synovus Bank   
2                      19248.0                           Synovus Bank   
3                       9551.0  Bank of America, National Association   
4                      19248.0                           Synovus Bank   
5                      57328.0           The Huntington National Bank   
6                      57328.0           The Huntington National Bank   
7                       9551.0  Bank of America, National Association   
8                      57328.0           The Huntington National Bank   
9                          NaN                                    NaN   

   OriginatingLenderCity OriginatingLe

In [8]:
PPP_combined.columns

Index(['LoanNumber', 'DateApproved', 'SBAOfficeCode', 'ProcessingMethod',
       'BorrowerName', 'BorrowerAddress', 'BorrowerCity', 'BorrowerState',
       'BorrowerZip', 'LoanStatusDate', 'LoanStatus', 'Term',
       'SBAGuarantyPercentage', 'InitialApprovalAmount',
       'CurrentApprovalAmount', 'UndisbursedAmount', 'FranchiseName',
       'ServicingLenderLocationID', 'ServicingLenderName',
       'ServicingLenderAddress', 'ServicingLenderCity', 'ServicingLenderState',
       'ServicingLenderZip', 'RuralUrbanIndicator', 'HubzoneIndicator',
       'LMIIndicator', 'BusinessAgeDescription', 'ProjectCity',
       'ProjectCountyName', 'ProjectState', 'ProjectZip', 'CD', 'JobsReported',
       'NAICSCode', 'Race', 'Ethnicity', 'UTILITIES_PROCEED',
       'PAYROLL_PROCEED', 'MORTGAGE_INTEREST_PROCEED', 'RENT_PROCEED',
       'REFINANCE_EIDL_PROCEED', 'HEALTH_CARE_PROCEED',
       'DEBT_INTEREST_PROCEED', 'BusinessType', 'OriginatingLenderLocationID',
       'OriginatingLender', 'Originatin

In [9]:
PPP_combined['Race'].unique()

array(['Unanswered', 'White', 'Black or African American', 'Asian',
       'American Indian or Alaska Native',
       'Native Hawaiian or Other Pacific Islander', 'Eskimo & Aleut',
       'Multi Group', 'Puerto Rican'], dtype=object)

In [10]:
PPP_combined['Ethnicity'].unique()

array(['Unknown/NotStated', 'Not Hispanic or Latino',
       'Hispanic or Latino'], dtype=object)

In [11]:
#Create new column for year of approval 
PPP_combined['YearApproved'] = PPP_combined['DateApproved'].str[-4:]

#Create new column for NAICS 6 digit in string format 
PPP_combined['NAICS_6'] = PPP_combined['NAICSCode'].apply(lambda y: str(y)[:6])

In [13]:
#Create new column identifying minority-owned businesses as identified by Race and Ethnicity columns
PPP_combined.loc[PPP_combined['Race'] == 'American Indian or Alaska Native', 'Minority'] = 'Yes'  
PPP_combined.loc[PPP_combined['Race'] == 'Asian', 'Minority'] = 'Yes'  
PPP_combined.loc[PPP_combined['Race'] == 'Black or African American', 'Minority'] = 'Yes'  
PPP_combined.loc[PPP_combined['Race'] == 'Hispanic', 'Minority'] = 'Yes'  
PPP_combined.loc[PPP_combined['Race'] == 'Multi Group', 'Minority'] = 'Yes'  
PPP_combined.loc[PPP_combined['Race'] == 'Puerto Rican', 'Minority'] = 'Yes' 
PPP_combined.loc[PPP_combined['Race'] == 'Native Hawaiian or Other Pacific Islander', 'Minority'] = 'Yes' 
PPP_combined.loc[PPP_combined['Race'] == 'Eskimo & Aleut', 'Minority'] = 'Yes' 
PPP_combined.loc[PPP_combined['Race'] == 'White', 'Minority'] = 'No'
PPP_combined.loc[PPP_combined['Race'] == 'Unanswered', 'Minority'] = 'Unanswered'
PPP_combined.loc[PPP_combined['Ethnicity'] == 'Hispanic or Latino', 'Minority'] = 'Yes'

In [ ]:
#Merge NAICS 4 digit with PPP data and remove the 4 digit code from IndustrySubsector name
PPP_combined_NAICS = pd.merge(PPP_combined, NAICS_codes, how='left', left_on='NAICS_6', right_on='2017NAICSCode')
PPP_combined_NAICS.loc[:,'NAICS_4'] = PPP_combined_NAICS['4DigitNAICS'].apply(lambda y: str(y)[:4])
PPP_combined_NAICS = PPP_combined_NAICS.drop(['2017NAICSCode','Changed2017','TRIIndustrySector','NewIndustry','4DigitNAICS'], axis=1)
PPP_combined_NAICS['IndustrySubsector'] = PPP_combined_NAICS['IndustrySubsector'].str[5:]
PPP_combined_NAICS.head()

In [ ]:
PPP_combined_NAICS['BorrowerState'].unique()

In [ ]:
PPP_combined_NAICS_CA = PPP_combined_NAICS[PPP_combined_NAICS['BorrowerState']=='CA']
PPP_combined_NAICS_CA.head()

In [ ]:
PPP_combined_NAICS_no_state = PPP_combined_NAICS[PPP_combined_NAICS['BorrowerState'].isna()]
PPP_combined_NAICS_no_state.head()

In [ ]:
PPP_combined_NAICS_CA_2020 = PPP_combined_NAICS_CA[PPP_combined_NAICS_CA['YearApproved']=='2020']
PPP_combined_NAICS_CA_2021 = PPP_combined_NAICS_CA[PPP_combined_NAICS_CA['YearApproved']=='2021']

In [ ]:
#Write to CSV
PPP_combined_NAICS_CA_2020.to_csv('PPP_extract_CA_20.csv')
PPP_combined_NAICS_CA_2021.to_csv('PPP_extract_CA_21.csv')

In [ ]:
#Write to CSV
PPP_combined_NAICS_no_state.to_csv('PPP_extract_no_state.csv')

## Summary Data Tables 

In [ ]:
PPP_ind_4 = pd.DataFrame(PPP_combined_NAICS.groupby(['NAICS_4','IndustrySubsector'])['JobsReported']
                          .agg(Businesses='count',Jobs='sum').sort_values(by='Businesses', ascending=False))
PPP_ind_4.reset_index(inplace=True)
PPP_ind_4.index = np.arange(1, len(PPP_ind_4) + 1)
PPP_ind_4.head(20)

In [ ]:
retail_4 = ['4411', '4412', '4413', '4421', '4422', '4431', '4441', '4442', '4451', '4452', '4453', '4461', '4471', '4481', 
            '4482', '4483', '4511', '4512', '4522', '4523', '4531', '4532', '4533', '4539', '4541', '4542', '4543']

In [ ]:
PPP_combined_NAICS['NAICS_4'].dtypes

In [ ]:
PPP_retail_4 = PPP_ind_4[PPP_ind_4['NAICS_4'].isin(retail_4)]
PPP_retail_4

In [ ]:
personal_4 = ['8121', '8122', '8123', '8129']

In [ ]:
PPP_personal_4 = PPP_ind_4[PPP_ind_4['NAICS_4'].isin(personal_4)]
PPP_personal_4

In [ ]:
food_4 = ['7223', '7224', '7225']

In [ ]:
PPP_food_4 = PPP_ind_4[PPP_ind_4['NAICS_4'].isin(food_4)]
PPP_food_4

In [ ]:
PPP_combined_minority = PPP_combined_NAICS[PPP_combined_NAICS['Minority']=='Yes']
PPP_combined_minority.head(10)

In [ ]:
PPP_minority_ind_4 = pd.DataFrame(PPP_combined_minority.groupby(['NAICS_4','IndustrySubsector'])['JobsReported']
                          .agg(Businesses='count',Jobs='sum').sort_values(by='Businesses', ascending=False))
PPP_minority_ind_4.reset_index(inplace=True)
PPP_minority_ind_4.index = np.arange(1, len(PPP_minority_ind_4) + 1)
PPP_minority_ind_4.head(20)

In [ ]:
PPP_minority_retail_4 = PPP_minority_ind_4[PPP_minority_ind_4['NAICS_4'].isin(retail_4)]
PPP_minority_retail_4

In [ ]:
PPP_combined_NAICS['YearApproved'].unique()

In [ ]:
#Extract for 2021 approvals only 
PPP_combined_2021 = PPP_combined_NAICS[PPP_combined_NAICS['YearApproved']=='2021']
PPP_combined_2021.head(10)

In [ ]:
PPP_ind_4_2021 = pd.DataFrame(PPP_combined_2021.groupby(['NAICS_4','IndustrySubsector'])['JobsReported']
                          .agg(Businesses='count',Jobs='sum').sort_values(by='Businesses', ascending=False))
PPP_ind_4_2021.reset_index(inplace=True)
PPP_ind_4_2021.index = np.arange(1, len(PPP_top_ind) + 1)
PPP_ind_4_2021.head(20)

In [ ]:
PPP_retail_4_2021 = PPP_ind_4_2021[PPP_ind_4_2021['NAICS_4'].isin(retail_4)]
PPP_retail_4_2021

In [ ]:
PPP_combined_minority_2021 = PPP_combined_2021[PPP_combined_2021['Minority']=='Yes']
PPP_combined_minority_2021.head(10)

In [ ]:
PPP_minority_ind_4_2021 = pd.DataFrame(PPP_combined_minority_2021.groupby(['NAICS_4','IndustrySubsector'])['JobsReported']
                          .agg(Businesses='count',Jobs='sum').sort_values(by='Businesses', ascending=False))
PPP_minority_ind_4_2021.reset_index(inplace=True)
PPP_minority_ind_4_2021.index = np.arange(1, len(PPP_minority_ind_4_2021) + 1)
PPP_minority_ind_4_2021.head(20)

In [ ]:
PPP_minority_retail_4_2021 = PPP_minority_ind_4_2021[PPP_minority_ind_4_2021['NAICS_4'].isin(retail_4)]
PPP_minority_retail_4_2021